# FCA

Technical Challenge for Data Science Candidates

This workbook loads a pickle file from the prior notebook.

Naive Model

In [1]:
import numpy as np
import pandas as pd
import math
import json

from os import path

from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn import ensemble
from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE

import scipy.stats as st
import statsmodels as sm
import statsmodels.api as smi
from sklearn import svm

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

from cycler import cycler
import matplotlib.pyplot as plt
import seaborn as sns

pd.__version__

'0.24.2'

In [2]:
# If you turn this feature on, you can display each result as it happens.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# this is the local Utility module and it reloaded each time.
from fca import Utility

In [4]:
%load_ext autoreload
%autoreload 1
%aimport fca

In [5]:
# My utility singleton.
i0 = Utility.instance()

## Modelling

There are some highly correlated features that might be confusing.
Build a naive model that uses all the features and refine it down.

Filter the dataset and then apply a selection of models.

In [6]:
# df2 = pd.read_pickle("coded.pickle")
df2 = pd.read_pickle("scaled.pickle")
df2.shape

(41188, 29)

In [7]:
# re-instate the boolean binary outcome
df2.y = df2.y > 0
df2.y;

In [8]:
## Low incidence rate may require some up-sampling.
np.sum(df2.y.values) / len(df2.y)

0.11265417111780131

### Reload the dataset for the model

You need to run this after you have chosen and evaluated to reset the dataset.
There is no scaling needed for the models chosen.
If you run all cells, the default is evaluate for the never-active with logistic regression.

### Splitting

I use sample0 for the different datasets. 
 1. use a train/test split
 2. use everything - let cross-validation make the splits

In [37]:
sample0=2

### Case 0: Prescient data

Null hypothesis.

I include the outcome variable - forcing over-fitting. All the models should converge to one. Or the data is structurally unsound (mis-scaled) or completely noise. I then remove each of the prescient variables.

In [20]:
y = df2.y
# X = df2.drop(columns=['y'])
X = df2

### Case 1: All features

Remove the outcome and get a baseline

In [38]:
y = df2.y
X = df2.drop(columns=['y'])
# X = df2
X.columns
X.shape

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'job0', 'marital0',
       'education0', 'default0', 'housing0', 'loan0', 'poutcome0', 'pdays0'],
      dtype='object')

(41188, 28)

### Case 2: case 1 and drop rows

Example with dropping rows on a feature. This is scaled data. It may already be cut in an earlier notebook in which the scaling will be out.

In [ ]:
fig, ax = plt.subplots(figsize=(18,6))
sns.violinplot(ax=ax, x=X['nr.employed'], y=y)

In [ ]:
# The decision tree  
# idxs = ~(X.pdays0 <= -2.461)
idxs = X['nr.employed'] > -0.5
sum(idxs.values) / len(idxs.values)

In [ ]:
X = X[idxs]
y = y[idxs]

### Case 3: case 1 and drop columns

In [ ]:
X = X.drop(columns=pcols)
X.columns

### Split the Data

In [39]:
Xcols = list(X.columns)
if sample0 == 1:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
elif sample0 == 2:
    X_train, X_test, y_train, y_test = ( X, None, y, None )

In [40]:
X_train.shape
y_train.shape
Xcols

(41188, 28)

(41188,)

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'job0',
 'marital0',
 'education0',
 'default0',
 'housing0',
 'loan0',
 'poutcome0',
 'pdays0']

## Model cases

Again choose one to apply to the data you've chosen.

These are a number of models that can be applied. Some are 'patchy', results have high variance. Some are good for spotting mal-formed features.

    1. The first SVC is not practical for "big" data (width and count), but works well for smaller ones.

    2. The Multi-Layer Perceptron Classifier is a useful neural network and can get reasonable results quickly - it parallizes well. The hidden_layer_sizes need tuning to get a cross validation score over 0.5. Can be patchy. Spots prescients.

    3. The logistic regression is a very solid performer - not patchy. Usually spots prescients.

    4. Gradient-boosted is fast and spots prescients.

    5. Random-forest usually gets balanced results
    
When the MLPC is patchy, it usually means there is a useful split to be made. There is a Decision Tree Classifier below that can help find the split.

In [ ]:
clf = svm.SVC(kernel='linear', C=1)

In [41]:
# Useful method. Reduce the size of the neural network to find a path with prescient features. log_2 should be minimal
x0 = math.floor(math.log(X.shape[1], 2))
clf =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(x0, x0, 2), 
                     random_state=1, warm_start=True)

In [49]:
original_params = {'n_estimators': 1000, 'max_leaf_nodes': 8, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 8}
params = original_params

clf = ensemble.GradientBoostingClassifier(**params)

In [61]:
clf = ensemble.RandomForestClassifier(max_depth=2, random_state=0)

In [67]:
clf = LogisticRegression(C=1e4, solver='lbfgs', multi_class='auto', max_iter=10000) # max_iter is 2 orders up!

In [55]:
# Useful for plotting see below
# This can be compared with the MLCP if we have patchy validation results.
clf = tree.DecisionTreeClassifier()

## From cross-validation results, choose the most accurate model

To evaluate the model under a cross-validation scheme, run this cell.

In [68]:
scores = cross_val_score(clf, X_train, y_train, cv=5, error_score=np.nan)
scores
X_train.columns

array([0.89366351, 0.87169216, 0.88528769, 0.77042613, 0.65982761])

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'job0', 'marital0',
       'education0', 'default0', 'housing0', 'loan0', 'poutcome0', 'pdays0'],
      dtype='object')

## Confusion Matrix

If sample0 is 1 look at the confusion matrix

In [41]:
def perf0(X, y):
    global clf
    clf = clf.fit(X, y)
    y_pred = clf.predict(X)
    return (metrics.accuracy_score(y, y_pred), metrics.confusion_matrix(y, y_pred))

In [42]:
perf0(X_train, y_train)

(0.9105918057663126, array([[28490,   768],
        [ 2178,  1514]]))

In [46]:
a0, conf0 = perf0(X_test, y_test)
conf0
tn, fp, fn, tp = conf0.ravel()
(tn, fp, fn, tp)

array([[7107,  183],
       [ 543,  405]])

(7107, 183, 543, 405)

## Feature selection

Try to determine best features

In [69]:
clf # remind ourselves which classifier we're using.

LogisticRegression(C=10000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
# create the RFE model and select 3 attributes
rfe = RFE(clf, 5)
rfe = rfe.fit(X_train, y_train); # suppress messages Future Warning
# summarize the selection of the attributes
rfe.support_
rfe.ranking_

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False,  True,  True, False, False,
       False,  True, False, False, False, False, False, False,  True,
       False])

array([15, 18, 14, 10, 13, 22, 19,  4,  3, 12,  1,  9,  6, 16,  1,  1,  5,
        8,  2,  1, 24, 23, 11,  7, 20, 21,  1, 17])

In [71]:
x0 = np.array(Xcols)
f0s = list(x0[rfe.support_]) # just the first choices
f0s

['duration', 'poutcome', 'emp.var.rate', 'nr.employed', 'poutcome0']

In [72]:
# And in order
rnk0 = pd.DataFrame(data={'name': list(X_train.columns), 'rank': rfe.ranking_})
rnk0.sort_values(['rank'])

,name,rank
15,emp.var.rate,1
19,nr.employed,1
10,duration,1
26,poutcome0,1
14,poutcome,1
18,euribor3m,2
8,month,3
7,contact,4
16,cons.price.idx,5
12,pdays,6


In [ ]:
idxs = list(np.where(rfe.support_)[0])
idxs

### Decision Tree Classifier

This is hard work for scikit-learn

In [ ]:
# Train Decision Tree Classifer

cols = X_train.columns
cols = list(set(cols).difference(f0s))
cols

clf = clf.fit(X_train[cols], y_train)
y_pred = clf.predict(X_train[cols])
metrics.accuracy_score(y_train, y_pred)

In [ ]:
tree.plot_tree(clf, max_depth=4, feature_names = cols, class_names=['0','1'])

In [ ]:
dot_data = 'tree.dot'
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, max_depth=4, feature_names = cols, class_names=['0','1'])

# Summary

Case 1. Never active. The models all fit quickly and over cross-validation manage to get to over 90% accuracy.
Only MLPC struggles.

Case 2. Inactive for over a year. SVM really struggles to converge. Logistic regression needs 10000 iterations but gets to over 90%. MLPC gets there but is probably over-fitting.

Case 3. Inactive for over a month. MLCP gets 30% with full layers. Logistic regression can get to over 75%.

Running through the models, and looking at the feature selection, it seems that, for all cases, if a user has had transaction declined, or has had one reverted or declared as pending, then they prove to be inactive. For some reason a notification of ONBOARDING_TIPS_ACTIVATED_USERS seems to be a good indicator. This might be that account-holders who are having trouble ask for help in the form of onboarding tips.